In [2]:
!pip install obspy

  Obtaining dependency information for obspy from https://files.pythonhosted.org/packages/a0/12/8a5619d724e60a952403567e527949f2f660233c4720a9a405712de14df4/obspy-1.4.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.1 MB 435.7 kB/s eta 0:00:33
   ---------------------------------------- 0.1/14.1 MB 751.6 kB/s eta 0:00:19
   ---------------------------------------- 0.2/14.1 MB 984.6 kB/s eta 0:00:15
    --------------------------------------- 0.2/14.1 MB 1.2 MB/s eta 0:00:12
    --------------------------------------- 0.2/14.1 MB 1.2 MB/s eta 0:00:12
    --------------------------------------- 0.2/14.1 MB 1.2 MB/s eta 0:00:12
   - -------------------------------------- 0.4/14.1 MB 1.1 MB/s eta 0:00:13
   - -------------------------------------- 0.4/14.1 MB 1.1 MB/s eta 0:00:13
   - -----------------------------

In [3]:
from obspy.core import read, UTCDateTime
import os
import sys
import glob
import datetime
import warnings
import numpy as np
from scipy.signal import fftconvolve
from scipy.ndimage import uniform_filter

def WvfmSimilarity(st_data, st_templates, Tshift):
    """ Кросс-корреляция между шаблонами и данными. """
    ids = [tr.id for tr in st_templates]
    cc = None
    for id_ in ids:
        tr1 = st_data.select(id=id_)[0]
        tr2 = st_templates.select(id=id_)[0]
        if len(tr1) > len(tr2):
            data_short = tr2.data
            data_long = tr1.data
        else:
            data_short = tr1.data
            data_long = tr2.data

        corr_top = fftconvolve((data_long - np.mean(data_long)), (data_short - np.mean(data_short))[::-1], mode='valid')
        lds = len(data_short)
        ldl = len(data_long)
        ld = ldl - lds + 1
        val_a = uniform_filter(data_long, lds, mode='constant')
        val_b = uniform_filter(data_long * data_long, lds, mode='constant')
        sind = int(np.floor((lds + 1) / 2))
        std_dl = np.sqrt(val_b - val_a * val_a)[lds - sind:-sind + 1]
        std_ds = data_short.std()
        corr_bot = std_dl * std_ds * lds
        cor_coef = corr_top / corr_bot

        try:
            cc += cor_coef
        except TypeError:
            cc = cor_coef
        except ValueError:
            cc = None
            break

    if cc is None:
        warnings.warn('No valid data-template pairs. Correlation has not run.')
        return None, None
    time_beg = UTCDateTime(st_data[0].stats.starttime)
    time_mod = time_beg + Tshift
    st_delta = st_data[0].stats.delta
    orig_t = [(time_mod + (st_delta * ind)) for ind in range(0, len(cc), 1)]
    return np.array(cc) / len(ids), orig_t

def read_and_process_data(filename):
    """ Чтение и предварительная обработка данных из файла MiniSEED. """
    st = read(filename)
    st.detrend(type='demean')
    st.filter('bandpass', freqmin=1, freqmax=20, corners=4, zerophase=True)
    return st

def main():
    file_path = r"C:\Users\NeKonn\orders\cross-correlation\seed_primer\12.mseed"
    st_data = read_and_process_data(file_path)

    # Примерный шаблон (используем один и тот же поток для упрощения)
    st_template = st_data.copy()
    Tshift = 10  # Примерное смещение в секундах

    # Вызов функции кросс-корреляции
    cc, times = WvfmSimilarity(st_data, st_template, Tshift)
    if cc is not None:
        print("Cross-correlation coefficients:", cc)
        print("Estimated origin times:", times)
    else:
        print("No valid correlations were performed.")

if __name__ == "__main__":
    main()

Cross-correlation coefficients: [1.]
Estimated origin times: [UTCDateTime(2018, 6, 1, 6, 0, 10)]


In [6]:
!pip install --upgrade obspy